In [ ]:
''' !git clone https://github.com/Han-JaeHoon/QRNN-for-Sequential-Classification.git
!pip install pykan
!pip install pennylane
%cd QRNN-for-Sequential-Classification '''

In [1]:
# Quantum
import pennylane as qml
# PyTorch
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
# Numpy, Pandas
import numpy as np
import pandas as pd
# Layer
from kan import KAN
from RNN_block import RNN_block
# Data processing
from fucntions import data_seq, train_seq
from sklearn.preprocessing import MinMaxScaler
# Quantum User-Def Classes
from utils import my_utils
from NQE_class import NQE
from NQE_train_class import NQE_Train

### Settings

## Data uploading

In [4]:
locations = ['Adelaide', 'Albany', 'Albury', 'AliceSprings', 'BadgerysCreek', 'Ballarat', 'Bendigo', 'Brisbane', 'Cairns', 'Canberra', 'Cobar', 'CoffsHarbour', 'Dartmoor', 'Darwin', 'GoldCoast', 'Hobart', 'Katherine', 'Launceston', 'Melbourne', 'MelbourneAirport', 'Mildura', 'Moree', 'MountGambier', 'MountGinini', 'Newcastle', 'Nhil', 'NorahHead', 'NorfolkIsland', 'Nuriootpa', 'PearceRAAF', 'Penrith', 'Perth', 'PerthAirport', 'Portland', 'Richmond', 'Sale', 'SalmonGums', 'Sydney', 'SydneyAirport', 'Townsville', 'Tuggeranong', 'Uluru', 'WaggaWagga', 'Walpole', 'Watsonia', 'Williamtown', 'Witchcliffe', 'Wollongong', 'Woomera']

In [5]:
nqe_train = 1000

In [6]:
train_data_dict = dict()
label_data_dict = dict()
for e in locations:
    print(e)
    train_df = pd.read_csv("./data/train_data_" + e + ".csv")
    label_df = pd.read_csv("./data/label_data_" + e + ".csv")
    train_data_dict[e] = torch.tensor(train_df[["MinTemp","MaxTemp","Rainfall","Humidity3pm","Pressure3pm"]].to_numpy()[:nqe_train]).to(torch.float)
    label_data_dict[e] = torch.tensor(label_df['RainTomorrow'].to_numpy()[:nqe_train]).to(torch.float)


Adelaide
Albany
Albury
AliceSprings
BadgerysCreek
Ballarat
Bendigo
Brisbane
Cairns
Canberra
Cobar
CoffsHarbour
Dartmoor
Darwin
GoldCoast
Hobart
Katherine
Launceston
Melbourne
MelbourneAirport
Mildura
Moree
MountGambier
MountGinini
Newcastle
Nhil
NorahHead
NorfolkIsland
Nuriootpa
PearceRAAF
Penrith
Perth
PerthAirport
Portland
Richmond
Sale
SalmonGums
Sydney
SydneyAirport
Townsville
Tuggeranong
Uluru
WaggaWagga
Walpole
Watsonia
Williamtown
Witchcliffe
Wollongong
Woomera


In [7]:
nqe_x_train = train_data_dict[locations[0]]
nqe_y_train = label_data_dict[locations[0]]


non_zero = (nqe_y_train != 0).nonzero()

nqe_x_train = torch.squeeze(train_data_dict[locations[0]][non_zero])
nqe_y_train = torch.squeeze(label_data_dict[locations[0]][non_zero])



import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(nqe_x_train, nqe_y_train, test_size = 0.2, random_state = 42)

# 데이터 정규화
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Tensor로 변환
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_7564\3772276789.py:29: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_7564\3772276789.py:31: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_test_tensor = torch.tensor(y_test, dtype=torch.float32)


In [8]:
nqe_train_list = []
nqe_train_label_list = []
nqe_test_list = []
nqe_test_label_list = []

for i in range(len(X_train_tensor)):
    nqe_train_data = torch.stack([X_train_tensor, torch.concat([X_train_tensor[(i + 1) : ], X_train_tensor[ : (i + 1)]])])
    nqe_train_list.append(nqe_train_data)
    nqe_label_data = torch.stack([y_train_tensor,torch.concat([y_train_tensor[(i + 1) :],y_train_tensor[: (i + 1)]])])
    nqe_train_label_list.append(nqe_label_data)
  
for i in range(len(X_test_tensor)):
    nqe_test_data = torch.stack([X_test_tensor, torch.concat([X_test_tensor[(i + 1) : ], X_test_tensor[ : (i + 1)]])])
    nqe_test_list.append(nqe_test_data)
    nqe_label_data = torch.stack([y_test_tensor,torch.concat([y_test_tensor[(i + 1) :],y_test_tensor[: (i + 1)]])])
    nqe_test_label_list.append(nqe_label_data)

nqe_train_data = torch.concat(nqe_train_list, dim = 1)
nqe_train_label = torch.concat(nqe_train_label_list, dim = 1)
nqe_test_data = torch.concat(nqe_test_list, dim = 1)
nqe_test_label = torch.concat(nqe_test_label_list, dim = 1)


print(nqe_train_data.shape)
print(nqe_train_label.shape)
print(nqe_test_data.shape)
print(nqe_test_label.shape)


nqe_train_data = nqe_train_data.permute(1,0,2)
nqe_test_data = nqe_test_data.permute(1,0,2)
nqe_train_label = nqe_train_label.permute(1,0)
nqe_test_label = nqe_test_label.permute(1,0)

print(nqe_train_data.shape)
print(nqe_train_label.shape)
print(nqe_test_data.shape)
print(nqe_test_label.shape)


from torch.utils.data import DataLoader, TensorDataset
train_dataset = TensorDataset(nqe_train_data, nqe_train_label)
test_dataset = TensorDataset(nqe_test_data, nqe_test_label)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

torch.Size([2, 603729, 5])
torch.Size([2, 603729])
torch.Size([2, 38025, 5])
torch.Size([2, 38025])
torch.Size([603729, 2, 5])
torch.Size([603729, 2])
torch.Size([38025, 2, 5])
torch.Size([38025, 2])


In [17]:
nqe_train_label

tensor([[-1., -1.],
        [-1., -1.],
        [-1., -1.],
        ...,
        [ 1.,  1.],
        [-1., -1.],
        [-1., -1.]])

## NQE Train

In [18]:
import sys
def criterion(pred, label):
    '''
        pred : inner product of two states
        label : label data
    '''
    # print(pred.shape)
    # print(label.shape)
    loss = torch.sum(((pred) - 0.5 * (label[:, 0] * label[:, 1] + 1)) ** 2 ) / len(pred)
    return loss

def accuarcy(pred, label):
    '''
        pred : inner product of two states
        label : label data
    '''
    acc = torch.sum((torch.round(pred) == torch.round(0.5 * (label[:, 0] * label[:, 1] + 1)))) / len(pred)
    return acc

In [10]:
# nqe1.load_state_dict(torch.load("./models/nqe_fc_loss284.pth"))

In [19]:
n_qu = 5 # number of features
nqe1 = NQE(n_qu, 'FC')
nqe_train1 = NQE_Train(nqe1, criterion, train_loader,test_loader,[accuarcy])

In [20]:
trained_nqe = nqe_train1.train(15)

 loss : 0.06449 metric0 : 0.94118

 train_loss : 0.06449 test_loss : 0.27944 train_metric : 0.94118 test_metric : 0.68032

 loss : 0.00270 metric0 : 1.00000

 train_loss : 0.00270 test_loss : 0.23277 train_metric : 1.00000 test_metric : 0.73333

 loss : 0.05036 metric0 : 0.88235

 train_loss : 0.05036 test_loss : 0.22562 train_metric : 0.88235 test_metric : 0.74496

 loss : 0.15122 metric0 : 0.82353

 train_loss : 0.15122 test_loss : 0.24203 train_metric : 0.82353 test_metric : 0.72082

 loss : 0.05250 metric0 : 0.94118

 train_loss : 0.05250 test_loss : 0.24449 train_metric : 0.94118 test_metric : 0.70619

 loss : 0.09443 metric0 : 0.88235

 train_loss : 0.09443 test_loss : 0.22301 train_metric : 0.88235 test_metric : 0.73449

 loss : 0.11897 metric0 : 0.88235

 train_loss : 0.11897 test_loss : 0.22020 train_metric : 0.88235 test_metric : 0.74012

 loss : 0.00382 metric0 : 1.00000

 train_loss : 0.00382 test_loss : 0.23783 train_metric : 1.00000 test_metric : 0.73260

 loss : 0.05970 

---

In [21]:
torch.save(trained_nqe.state_dict(), "./models/nqe_fc_loss220.pth")

In [26]:
nqe2 = NQE(n_qu, 'KAN')
nqe_train2 = NQE_Train(nqe2, criterion, train_loader, test_loader, [accuarcy])

In [27]:
trained_nqe2 = nqe_train2.train(10)
torch.save(trained_nqe2.state_dict(), "./models/nqe_kan_0815.pth")

 loss : 0.19757 metric0 : 0.71875

RuntimeError: [enforce fail at alloc_cpu.cpp:114] data. DefaultCPUAllocator: not enough memory: you tried to allocate 101376 bytes.

In [19]:
torch.save(trained_nqe2.state_dict(), "./models/nqe_kan_loss291.pth")

: 